In [1]:
%matplotlib inline
import mqbench,torch,torchvision,numpy as np,matplotlib.pyplot as plt,torchvision,torchvision.models as models,timm,timm.models as models,torch.nn as nn
from torchmetrics import ConfusionMatrix
from plot import plot2dicts
from mqbench.prepare_by_platform import prepare_by_platform
from mqbench.prepare_by_platform import BackendType
from mqbench.utils.state import enable_calibration
from mqbench.utils.state import enable_quantization
from mqbench.convert_deploy import convert_deploy
from tqdm import tqdm
from mqbench.utils.logger import logger as log
from mqbench.fake_quantize.lsq import LearnableFakeQuantize
from dataset import get_dataloader
from timm.models.swin_transformer import SwinTransformer
from logger import get_logger
from mqbench.utils.registry import DEFAULT_MODEL_QUANTIZER
from torch.fx.graph_module import GraphModule
import timm.models.swin_transformer as st
from timm.models.swin_transformer import SwinTransformer
print(DEFAULT_MODEL_QUANTIZER)
logger,workdir = get_logger("softmax_quant")
log = logger # 

device = torch.device('cuda')
st.M = st.M.to(device)
mean=np.array([123.675, 116.28, 103.53])/255
std=np.array([58.395, 57.12, 57.375])/255

# dataloader = get_dataloader() #
extra_qconfig_dict = {
    'w_observer': 'MinMaxObserver',
    'a_observer': 'EMAMinMaxObserver',
    # 'a_observer': 'MSEObserver',
    'w_fakequantize': 'FixedFakeQuantize',
    'a_fakequantize': 'FixedFakeQuantize',
    # 'a_fakequantize': 'LearnableFakeQuantize',
    # 'a_fakequantize': 'FixedFakeQuantize',
    'w_qscheme': {
        'bit': 8,
        'symmetry': True,
        'per_channel': True,
        'pot_scale': False
    },
    'a_qscheme': {
        'bit': 4,
        'symmetry': True,
        'per_channel': False,
        'pot_scale': False
    }
}
extra_quantizer_dict = {
    'additional_module_type':(nn.Softmax,) # 额外需要量化的模型
}
# extra_fuse_dict = { # 用来完成qat模型的替换
#     # 'additional_qat_module_mapping':{nn.Softmax:MySoftmaxQAT}
# }
logger.info(extra_qconfig_dict) #  
model_name = 'swin_base_patch4_window7_224'
model = timm.create_model('swin_base_patch4_window7_224',pretrained=True).to(device) # 创建模型
# model = timm.create_model('resnet18',pretrained=True).to(device) # 创建模型
# prepare_custom_config_dict = {'extra_qconfig_dict': extra_qconfig_dict}
prepare_custom_config_dict = {'extra_qconfig_dict': extra_qconfig_dict,'leaf_module':[nn.Softmax],'extra_quantizer_dict':extra_quantizer_dict}
model = prepare_by_platform(model, BackendType.Tensorrt,prepare_custom_config_dict).to(device)
ori= timm.create_model('swin_base_patch4_window7_224',pretrained=True).to(device) #
model.eval() # 进行PTQ
enable_calibration(model) # 打开校准
dataloader = get_dataloader() #
with torch.no_grad():
    for i,(img,label) in enumerate(tqdm(dataloader)):
        if i>=32:
            break
        img = img.to(device)
        model(img)

enable_quantization(model) # 打开量化，准备好模拟后台推断的量化
ori.to(device)
ori.eval()
from torchmetrics import Accuracy

from collections import defaultdict
d_ori = defaultdict(list)
d_quant = defaultdict(list)
def hook_generator(d,name):
    def hook(module,input,output):
        if module.has_mask:
        # if input[1] is not None:
        #     name_ = name+'with mask'
            d[name+' (with mask)'].append((input[0].detach().cpu().numpy(),output.detach().cpu().numpy()))
        else:
            d[name].append((input[0].detach().cpu().numpy(),output.detach().cpu().numpy()))
        # d[name+'_output'].append(output.detach().cpu().numpy())
        return output
    return hook

for name,m in ori.named_modules():
    if name.endswith("softmax"):
        m.register_forward_hook(hook_generator(d_ori,name))

for name,m in model.named_modules():
    if name.endswith("softmax"):
        m.register_forward_hook(hook_generator(d_quant,name))

# model = ori
acc = Accuracy()
acc_ori = Accuracy()
with torch.no_grad():
    for i,(img,label) in enumerate(tqdm(dataloader)):
        if i>2:
            break
        img = img.to(device)
        # acc.update(torch.argmax(model(img),dim=-1),torch.argmax(ori(img),dim=-1))
        acc.update(model(img).detach().cpu(),label.cpu())
        acc_ori.update(ori(img).detach().cpu(),label.cpu())
logger.info(f"最终的精度是：{acc.compute()}")
logger.info(f"ori_最终的精度是：{acc_ori.compute()}")

softmax_quantLog创建完成


OrderedDict()


{'w_observer': 'MinMaxObserver', 'a_observer': 'EMAMinMaxObserver', 'w_fakequantize': 'FixedFakeQuantize', 'a_fakequantize': 'FixedFakeQuantize', 'w_qscheme': {'bit': 8, 'symmetry': True, 'per_channel': True, 'pot_scale': False}, 'a_qscheme': {'bit': 4, 'symmetry': True, 'per_channel': False, 'pot_scale': False}}


[MQBENCH] INFO: Quantize model Scheme: BackendType.Tensorrt Mode: Training
[MQBENCH] INFO: Weight Quant Scheme is overrided!
[MQBENCH] INFO: Activation Quant Scheme is overrided!
[MQBENCH] INFO: Weight Qconfig:
    FakeQuantize: FixedFakeQuantize Params: {}
    Oberver:      MinMaxObserver Params: Symmetric: True / Bitwidth: 8 / Per channel: True / Pot scale: False / Extra kwargs: {}
[MQBENCH] INFO: Activation Qconfig:
    FakeQuantize: FixedFakeQuantize Params: {}
    Oberver:      EMAMinMaxObserver Params: Symmetric: True / Bitwidth: 4 / Per channel: False / Pot scale: False / Extra kwargs: {}
[MQBENCH] INFO: Replace module to qat module.
[MQBENCH] INFO: Insert act quant add_1_post_act_fake_quantizer
[MQBENCH] INFO: Insert act quant view_15_post_act_fake_quantizer
[MQBENCH] INFO: Insert act quant add_10_post_act_fake_quantizer
[MQBENCH] INFO: Insert act quant view_37_post_act_fake_quantizer
[MQBENCH] INFO: Insert act quant add_21_post_act_fake_quantizer
[MQBENCH] INFO: Insert act qua

  2%|▏         | 32/1563 [00:14<11:20,  2.25it/s] 

[MQBENCH] INFO: Disable observer and Enable quantize.



  0%|          | 3/1563 [00:13<2:01:12,  4.66s/it]
最终的精度是：0.9583333134651184
ori_最终的精度是：0.9895833134651184


In [3]:
# plot2dicts(d_ori,d_quant)
# for n,m in model.named_modules():
#     if n.endswith('quantizer'):
#         display(n)
#         display(m.activation_post_process.min_val,m.activation_post_process.max_val)
d_ori['layers.0.blocks.1.attn.softmaxwith mask']

[(array([[[[ 3.07728839e+00,  1.55387902e+00,  1.42091417e+00, ...,
             1.23994327e+00, -9.50953186e-01, -1.09436834e+00],
           [-6.41657710e-02,  1.85206652e+00,  1.26256573e+00, ...,
            -3.10189605e-01, -1.62719178e+00, -1.62522602e+00],
           [-1.11304283e+00,  6.81161880e-04,  2.70337486e+00, ...,
            -4.23293799e-01, -2.50727081e+00, -1.38531530e+00],
           ...,
           [ 5.28361082e-01,  1.80833399e-01,  7.84095943e-01, ...,
             1.59389782e+00, -6.53180540e-01, -9.68959570e-01],
           [-8.44753742e-01, -3.44158053e-01, -2.09742963e-01, ...,
             9.99661982e-02,  1.66459894e+00,  7.96519637e-01],
           [-1.78802252e+00, -1.73611331e+00, -1.49747074e-01, ...,
            -8.75039577e-01, -3.32164705e-01,  1.87095451e+00]],
  
          [[ 1.26613533e+00,  3.97487187e+00,  2.19619131e+00, ...,
            -2.00005627e+00, -2.06459093e+00, -1.46712267e+00],
           [ 4.80597115e+00, -2.22095400e-02,  4.1255860

## 帮助统计 Softmax的输入scale

In [1]:

%matplotlib inline
import mqbench,torch,torchvision,numpy as np,matplotlib.pyplot as plt,torchvision,torchvision.models as models,timm,timm.models as models,torch.nn as nn
from torchmetrics import ConfusionMatrix
from plot import plot2dicts
from mqbench.prepare_by_platform import prepare_by_platform
from mqbench.prepare_by_platform import BackendType
from mqbench.utils.state import enable_calibration
from mqbench.utils.state import enable_quantization
from mqbench.convert_deploy import convert_deploy
from tqdm import tqdm
from mqbench.utils.logger import logger as log
from mqbench.fake_quantize.lsq import LearnableFakeQuantize
from dataset import get_dataloader
from timm.models.swin_transformer import SwinTransformer
from logger import get_logger
from mqbench.utils.registry import DEFAULT_MODEL_QUANTIZER
from torch.fx.graph_module import GraphModule
import timm.models.swin_transformer as st
print(DEFAULT_MODEL_QUANTIZER)
# model_name = 'swin_bas_patch4_window7_224'
# model_name = 'vit_base_patch8_224'
# model_name = 'swinv2_base_window8_256'
# model_name = 'deit_base_patch16_224'
model_name = 'volo_d1_224'
logger,workdir = get_logger("softmax_quant")
log = logger # 

device = torch.device('cuda')
st.M = st.M.to(device)
mean=np.array([123.675, 116.28, 103.53])/255
std=np.array([58.395, 57.12, 57.375])/255

# dataloader = get_dataloader() #
extra_qconfig_dict = {
    'w_observer': 'MinMaxObserver',
    'a_observer': 'EMAMinMaxObserver',
    # 'a_observer': 'MSEObserver',
    'w_fakequantize': 'FixedFakeQuantize',
    'a_fakequantize': 'FixedFakeQuantize',
    # 'a_fakequantize': 'LearnableFakeQuantize',
    # 'a_fakequantize': 'FixedFakeQuantize',
    'w_qscheme': {
        'bit': 8,
        'symmetry': True,
        'per_channel': True,
        'pot_scale': False
    },
    'a_qscheme': {
        'bit': 8,
        'symmetry': True,
        'per_channel': False,
        'pot_scale': False
    }
}
extra_quantizer_dict = {
    'additional_module_type':(nn.Softmax,) # 额外需要量化的模型
}
# extra_fuse_dict = { # 用来完成qat模型的替换
#     # 'additional_qat_module_mapping':{nn.Softmax:MySoftmaxQAT}
# }
logger.info(extra_qconfig_dict) #  
model = timm.create_model(model_name,pretrained=True).to(device) # 创建模型
# model = timm.create_model('resnet18',pretrained=True).to(device) # 创建模型
# prepare_custom_config_dict = {'extra_qconfig_dict': extra_qconfig_dict}
prepare_custom_config_dict = {'extra_qconfig_dict': extra_qconfig_dict,'leaf_module':[nn.Softmax],'extra_quantizer_dict':extra_quantizer_dict}
model = prepare_by_platform(model, BackendType.Tensorrt,prepare_custom_config_dict).to(device)
ori= timm.create_model(model_name,pretrained=True).to(device) #
model.eval() # 进行PTQ
enable_calibration(model) # 打开校准
dataloader = get_dataloader() #
with torch.no_grad():
    for i,(img,label) in enumerate(tqdm(dataloader)):
        if i>=32:
            break
        img = img.to(device)
        model(img)

enable_quantization(model) # 打开量化，准备好模拟后台推断的量化
ori.to(device)
ori.eval()
from torchmetrics import Accuracy

from collections import defaultdict
d_ori = defaultdict(list)
d_quant = defaultdict(list)
def hook_generator(d,name):
    def hook(module,input,output):
        d[name].append((input[0].detach().cpu().numpy(),output.detach().cpu().numpy()))
        # d[name+'_output'].append(output.detach().cpu().numpy())
        return output
    return hook

# for name,m in ori.named_modules():
#     if name.endswith("softmax"):
#         m.register_forward_hook(hook_generator(d_ori,name))

# for name,m in model.named_modules():
#     if name.endswith("softmax"):
#         m.register_forward_hook(hook_generator(d_quant,name))

softmax_list = list()
fq_list = list()
for name,m in model.named_modules():
    if name.endswith("softmax"):
        softmax_list.append(name)

    if name.endswith("_post_act_fake_quantizer"):
        fq_list.append(name)

# model = ori
acc = Accuracy()
acc_ori = Accuracy()
with torch.no_grad():
    for i,(img,label) in enumerate(tqdm(dataloader)):
        if i>10:
            break
        img = img.to(device)
        # acc.update(torch.argmax(model(img),dim=-1),torch.argmax(ori(img),dim=-1))
        acc.update(model(img).detach().cpu(),label.cpu())
        acc_ori.update(ori(img).detach().cpu(),label.cpu())
logger.info(f"最终的精度是：{acc.compute()}")
logger.info(f"ori_最终的精度是：{acc_ori.compute()}")

softmax_quantLog创建完成


OrderedDict()


{'w_observer': 'MinMaxObserver', 'a_observer': 'EMAMinMaxObserver', 'w_fakequantize': 'FixedFakeQuantize', 'a_fakequantize': 'FixedFakeQuantize', 'w_qscheme': {'bit': 8, 'symmetry': True, 'per_channel': True, 'pot_scale': False}, 'a_qscheme': {'bit': 8, 'symmetry': True, 'per_channel': False, 'pot_scale': False}}


RuntimeError: [enforce fail at inline_container.cc:145] . PytorchStreamReader failed reading zip archive: failed finding central directory

In [4]:
import pandas as pd
from timm.models.volo import VOLO

ret = pd.DataFrame(columns=['name','scale','min','max','qmin','qmax','observer'])

for name1,name2 in zip(fq_list,softmax_list):
    # display(getattr(model,name))
    m = getattr(model,name1)
    # display(name2)
    ret = ret.append(dict(name=name2,scale=m.scale.item(),min=m.activation_post_process.min_val.item(),max=m.activation_post_process.max_val.item(),qmin=m.quant_min,\
        qmax=m.quant_max,observer=type(m.activation_post_process).__name__),ignore_index=True)

import os.path as osp
ret.to_csv(model_name+'.csv')
ret
# assert len(fq_list) == len(softmax_list)
# for t in zip(softmax_list,fq_list):
#     print(t)
from timm.models.swin_transformer import SwinTransformer

,name,scale,min,max,qmin,qmax,observer
0,blocks.0.attn.softmax,0.171828,-19.260910,21.908123,-128,127,EMAMinMaxObserver
1,blocks.1.attn.softmax,0.161794,-14.856956,20.628693,-128,127,EMAMinMaxObserver
2,blocks.2.attn.softmax,0.114820,-14.174229,14.639574,-128,127,EMAMinMaxObserver
3,blocks.3.attn.softmax,0.343698,-43.821461,33.975819,-128,127,EMAMinMaxObserver
4,blocks.4.attn.softmax,0.177802,-22.669720,21.178539,-128,127,EMAMinMaxObserver
5,blocks.5.attn.softmax,0.105697,-11.184481,13.476347,-128,127,EMAMinMaxObserver
6,blocks.6.attn.softmax,0.105484,-11.533165,13.449196,-128,127,EMAMinMaxObserver
7,blocks.7.attn.softmax,0.095306,-12.151550,10.989811,-128,127,EMAMinMaxObserver
8,blocks.8.attn.softmax,0.105951,-13.508765,11.183628,-128,127,EMAMinMaxObserver
9,blocks.9.attn.softmax,0.103774,-11.608718,13.231167,-128,127,EMAMinMaxObserver


In [33]:
m = getattr(model,fq_list[0])
m.activation_post_process.min_val
m.quant_max
m.scale
type(m.activation_post_process).__name__

'EMAMinMaxObserver'

In [34]:
ret

,name,scale,min,max,qmin,qmax,ovserver
